In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,Date
0,USA,245442,565.0,6098.0,28.0,10411.0,228933,5421.0,742.0,18.0,2020-04-03 10:05:29.097543
1,Spain,117710,5645.0,10935.0,587.0,30513.0,76262,6416.0,2518.0,234.0,2020-04-03 10:05:29.097543
2,Italy,115242,0.0,13915.0,0.0,18278.0,83049,4053.0,1906.0,230.0,2020-04-03 10:05:29.097543
3,Germany,87244,2450.0,1138.0,31.0,24575.0,61531,3936.0,1041.0,14.0,2020-04-03 10:05:29.097543
4,France,59105,0.0,5387.0,0.0,12428.0,41290,6399.0,905.0,83.0,2020-04-03 10:05:29.097543
...,...,...,...,...,...,...,...,...,...,...,...
201,Caribbean Netherlands,2,0.0,0.0,0.0,0.0,2,0.0,76.0,0.0,2020-04-03 10:05:29.097543
202,Sierra Leone,2,0.0,0.0,0.0,0.0,2,0.0,0.3,0.0,2020-04-03 10:05:29.097543
203,Papua New Guinea,1,0.0,0.0,0.0,0.0,1,0.0,0.1,0.0,2020-04-03 10:05:29.097543
204,Timor-Leste,1,0.0,0.0,0.0,0.0,1,0.0,0.8,0.0,2020-04-03 10:05:29.097543


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
2,Italy,115242,0.0,13915.0,0.0,18278.0,83049,4053.0,1906.0,230.0,2020-04-03 10:05:29.097543,12.070013,42.796626,32193.0,0.432237,0.120746,0.116443,0.014060,Last 8 columns are added by Roaya
1,Spain,117710,5645.0,10935.0,587.0,30513.0,76262,6416.0,2518.0,234.0,2020-04-03 10:05:29.097543,-3.647550,40.244487,41448.0,0.263825,0.092898,0.149919,0.013927,Last 8 columns are added by Roaya
176,China,81620,31.0,3322.0,4.0,76571.0,1727,379.0,57.0,2.0,2020-04-03 10:05:29.097543,103.819074,36.561765,79893.0,0.041581,0.040701,0.288976,0.011762,Last 8 columns are added by Roaya
4,France,59105,0.0,5387.0,0.0,12428.0,41290,6399.0,905.0,83.0,2020-04-03 10:05:29.097543,2.618787,47.824905,17815.0,0.302386,0.091143,0.064438,0.005873,Last 8 columns are added by Roaya
5,Iran,53183,2715.0,3294.0,134.0,17935.0,31954,4035.0,633.0,39.0,2020-04-03 10:05:29.097543,54.274070,32.575033,21229.0,0.155165,0.061937,0.076786,0.004756,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Dominica,12,0.0,0.0,0.0,0.0,12,0.0,167.0,0.0,2020-04-03 10:05:29.097543,-61.357726,15.439470,0.0,0.000000,0.000000,0.000000,0.000000,Last 8 columns are added by Roaya
101,Georgia,148,14.0,0.0,0.0,27.0,121,6.0,37.0,0.0,2020-04-03 10:05:29.097543,43.507803,42.168558,27.0,0.000000,0.000000,0.000098,0.000000,Last 8 columns are added by Roaya
149,Greenland,10,0.0,0.0,0.0,3.0,7,0.0,176.0,0.0,2020-04-03 10:05:29.097543,-41.341911,74.710513,3.0,0.000000,0.000000,0.000011,0.000000,Last 8 columns are added by Roaya
150,Grenada,10,0.0,0.0,0.0,0.0,10,0.0,89.0,0.0,2020-04-03 10:05:29.097543,-61.682202,12.117250,0.0,0.000000,0.000000,0.000000,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"USA","TotalCases":245442,"NewCases":565.0,"TotalDeaths":6098.0,"NewDeaths":28.0,"TotalRecovered":10411.0,"ActiveCases":228933,"CriticalCases":5421.0,"Tot\\u00a0Cases\\/1M pop":742.0,"Deaths\\/1M pop":18.0,"Date":1585908329097,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":16509.0,"ClosedDeathRate":0.3693742807,"CurrentDeathRate":0.0248449736,"Weight":0.0597137473,"DeathRateXWeight":0.0014835865,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spain","TotalCases":117710,"NewCases":5645.0,"TotalDeaths":10935.0,"NewDeaths":587.0,"TotalRecovered":30513.0,"ActiveCases":76262,"CriticalCases":6416.0,"Tot\\u00a0Cases\\/1M pop":2518.0,"Deaths\\/1M pop":234.0,"Date":1585908329097,"Longitude":-3.64755047,"Latitude":40.24448698,"ClosedCases":41448.0,"ClosedDeathRate":0.2638245512,"CurrentDeathRate":0.0928977997,"Weight":0.1499191591,"DeathRateXWeight":0.01392716,"notes":"Last 8 columns are added by Roaya"},{"Country":"Italy","TotalCases":115242,"NewCases":0.0,"Tota